In [12]:
import pandas as pd
import numpy as np

path = '../../Data/'

players = []
for ii, year in enumerate(range(1979, 2019)):
    df = pd.read_csv(path + 'nba_' + str(year) + '_per_poss.csv')
    
    # Pull out total games for later
    g = df.G
    
    # Drop data not of interest
    df.drop(['Unnamed: 0', 'Rk', 'Pos', 'Age', 'Tm', 'G', 'GS'], axis = 1, inplace = True)
    
    # Use 1979 to define 1980 rookies as players not playing in 1979
    if year == 1979:b
        rookies = pd.DataFrame(columns = df.columns)
        mean_key = pd.DataFrame(columns = df.columns)
        mean_key.drop('Player', axis = 1, inplace = True)
        std_key = mean_key
        old_players = df.Player.values
        
    else:
        
        # Normalize each year's stats as (x - mu)/std. To reverse this, we need to save mu and std for each year
        mean_key = mean_key.append(df.agg(['mean']))
        std_key = std_key.append(df.agg(['std']))
        for jj in df.columns:
            if jj != 'Player':
                df[jj] = (df[jj] - df[jj].mean()) / df[jj].std()
                

        # Rookies defined as players whose names were not previously in players and who played half a season's games
        for jj, name in enumerate(df.Player):
            if name not in players and name not in old_players and g[jj] > np.max(g) // 2:
                players.append(name)
                rookies = rookies.append(df.loc[jj, ])
                rookies.loc[jj, 'Year'] = year
                
rookies.to_csv(path + 'nba_rookie_per_poss.csv')
mean_key.to_csv(path + 'nba_rookie_per_poss_meankey.csv')
std_key.to_csv(path + 'nba_rookie_per_poss_stdkey.csv')

In [5]:
import pandas as pd
import numpy as np

path = '../../Data/'

year = 1979
df = pd.read_csv(path + 'nba_' + str(year) + '_per_poss.csv')
df['Year'] = year
df.head()

,Unnamed: 0,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,...,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg,Year
0,0,1,Kareem Abdul-Jabbar*,C,31,LAL,80,NaN,3157,11.2,...,14.7,6.2,1.1,4.5,4.0,3.3,27.3,115.0,98,1979
1,1,2,Tom Abernethy,PF,24,GSW,70,NaN,1219,6.8,...,8.3,3.0,1.5,0.5,1.2,5.1,16.3,118.0,103,1979
2,2,3,Alvan Adams,C,24,PHO,77,NaN,2364,10.6,...,13.2,6.7,2.1,1.2,5.2,4.6,25.6,106.0,100,1979
3,3,4,Lucius Allen,PG,31,KCK,31,NaN,413,7.5,...,5.0,4.8,2.3,0.7,3.3,5.6,17.0,87.0,103,1979
4,4,5,Kim Anderson,SF,23,POR,21,NaN,224,5.0,...,9.3,3.1,0.8,1.0,4.6,8.7,13.1,75.0,105,1979
